# Determining the salary for player in FIFA 18

This dataset obtained from www.kaggle.com contains 185 fields for every player in FIFA 18:
 - Player info such as age, club, league, nationality, salary and physical attributes
 - All playing attributes, such as finishing and dribbling
 - Special attributes like skill moves and international reputation
 - Traits and specialities
 - Overall, potential, and ratings for each position
We would construct a regression model trying to predict the salary of each player. The idea covered in the textbook but not covered well in the lectures that we use is: 

Some of the columns contain special characters so we use UTF-8 encoding to read the .csv in properly. 

# Import and explore data

In [1]:
import pandas as pd
pd.options.display.max_columns = 50
df = pd.read_csv("complete.csv", encoding="utf-8")
df.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,weight_kg,body_type,real_face,flag,nationality,photo,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,...,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera División,1985-02-05,185.0,80.0,C. Ronaldo,True,https://cdn.sofifa.org/flags/38@3x.png,Portugal,https://cdn.sofifa.org/18/players/20801.png,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,...,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera División,1987-06-24,170.0,72.0,Messi,True,https://cdn.sofifa.org/flags/52@3x.png,Argentina,https://cdn.sofifa.org/18/players/158023.png,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,...,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,68.0,Neymar,True,https://cdn.sofifa.org/flags/54@3x.png,Brazil,https://cdn.sofifa.org/18/players/190871.png,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,176580,L. Suárez,Luis Suárez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera División,1987-01-24,182.0,86.0,Normal,True,https://cdn.sofifa.org/flags/60@3x.png,Uruguay,https://cdn.sofifa.org/18/players/176580.png,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,92.0,Normal,True,https://cdn.sofifa.org/flags/21@3x.png,Germany,https://cdn.sofifa.org/18/players/167495.png,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


# Observe
- The data has a lot of very detail features such as international reputation, work rate at attacking and defending, etc. 
- We drop columns that we think do not correlate and redundant with salary: ID, special, club_logo, flag, photo, nationality, league, full_name, birth_date.
- As goal keepers (gk) have a separate set of properties, we move them to the gk dataframe. The other player data move to data dataframe.
- We drop the null salary and release clause enties.
- Cristiano Ronaldo, L.Messi and L. Suárez have exceptional high salary because they have several uncountable value. Therefore, I will drop these 3.
- Change work_rate from scale Low, Medium, High to scale 1,2,3.
- Change boolean to scale 0,1.
- Change preferred foot from scale Right, Left to scale 0,1.
- Change the body type from scale Lean, Normal, Stocky to scale 1, 2, 3.
- In player dataframe, drop all goalkeeper skills.
(Viet lai cai nay va add them nhung cai can lam o day)

In [2]:
# Drop exceptions
# df.drop(['20801', '158023', '176580'], axis=0)

# Drop columns.
df = df.drop(['ID','special','club','club_logo','birth_date','flag','photo','real_face', 'prefers_gk', 'nationality', 'league','full_name'],axis =1)


# Drop zero salary row.
df = df[df['eur_wage']!=0]
df = df[df['eur_release_clause']!=0]

df.head(5)

,name,age,height_cm,weight_kg,body_type,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,work_rate_def,preferred_foot,crossing,finishing,heading_accuracy,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
0,Cristiano Ronaldo,32,185.0,80.0,C. Ronaldo,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,33,80,5,5,4,High,Low,Right,85,94,88,...,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,L. Messi,30,170.0,72.0,Messi,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,26,61,5,4,4,Medium,Medium,Left,77,95,71,...,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Neymar,25,175.0,68.0,Neymar,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,30,60,5,5,5,High,Medium,Right,75,89,62,...,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,L. Suárez,30,182.0,86.0,Normal,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,42,81,5,4,4,High,Medium,Right,77,94,77,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,M. Neuer,31,193.0,92.0,Normal,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,60,91,5,1,4,Medium,Medium,Right,15,13,25,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [3]:
# Change scale
df['work_rate_att'].replace({'High':3,'Medium':2,'Low':1},inplace = True)
df['work_rate_def'].replace({'High':3,'Medium':2,'Low':1},inplace = True)

In [4]:
# Change scale
start_column = df.columns.get_loc("1_on_1_rush_trait")
end_column = df.columns.get_loc("prefers_lcb") + 1
for i in range (start_column,end_column):
    a = df.iloc[:,i]
    a.replace({True:1, False:0}, inplace = True)
df.head(5)

,name,age,height_cm,weight_kg,body_type,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,work_rate_def,preferred_foot,crossing,finishing,heading_accuracy,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
0,Cristiano Ronaldo,32,185.0,80.0,C. Ronaldo,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,33,80,5,5,4,3,1,Right,85,94,88,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,L. Messi,30,170.0,72.0,Messi,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,26,61,5,4,4,2,2,Left,77,95,71,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Neymar,25,175.0,68.0,Neymar,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,30,60,5,5,5,3,2,Right,75,89,62,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,L. Suárez,30,182.0,86.0,Normal,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,42,81,5,4,4,3,2,Right,77,94,77,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M. Neuer,31,193.0,92.0,Normal,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,60,91,5,1,4,2,2,Right,15,13,25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df['body_type'].value_counts()

Normal        10422
Lean           6145
Stocky         1168
C. Ronaldo        1
Courtois          1
Neymar            1
Messi             1
Akinfenwa         1
Shaqiri           1
Name: body_type, dtype: int64

In [6]:
# Change scale. Because Courtois, Ronaldo, Shaqiri, Akinfenwa, Messi, Neymar are all of good player, assume they are in type 1.
df['body_type'].replace({'Lean':1,'Courtois':1,'C. Ronaldo':1,'Shaqiri':1,'Akinfenwa':1,'Messi':1,'Neymar':1,'Normal':2,'Stocky':3,},inplace = True)
df['body_type'].value_counts()

2    10422
1     6151
3     1168
Name: body_type, dtype: int64

In [7]:
# Change scale
df['preferred_foot'].replace({'Left':1,'Right':0},inplace = True)
df['preferred_foot'].value_counts()

0    13547
1     4194
Name: preferred_foot, dtype: int64

In [8]:
# Put gk data to gk dataframe.
gk = df[df['gk'].notnull()]

# And drop all the null columns (properties not correlated with a goal keeper salary) in gk.
rs_column = gk.columns.get_loc("rs")
lcb_column = gk.columns.get_loc("lcb") + 1
gk = gk.drop(gk.columns[rs_column:lcb_column],axis=1)

# Finally, we drop the gk column in the df dataframe as not gk players do not have this property.
data = df.drop(['gk','gk_diving','gk_handling','gk_kicking','gk_positioning', 'gk_reflexes'],axis =1)
data.dropna()
data.head(5)

,name,age,height_cm,weight_kg,body_type,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,work_rate_def,preferred_foot,crossing,finishing,heading_accuracy,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
0,Cristiano Ronaldo,32,185.0,80.0,1,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,33,80,5,5,4,3,1,0,85,94,88,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,L. Messi,30,170.0,72.0,1,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,26,61,5,4,4,2,2,1,77,95,71,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Neymar,25,175.0,68.0,1,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,30,60,5,5,5,3,2,0,75,89,62,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,L. Suárez,30,182.0,86.0,2,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,42,81,5,4,4,3,2,0,77,94,77,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M. Neuer,31,193.0,92.0,2,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,60,91,5,1,4,2,2,0,15,13,25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
data.corr()['eur_wage'].sort_values(ascending = False)[:20]

eur_wage                    1.000000
eur_release_clause          0.852157
eur_value                   0.851180
international_reputation    0.704308
overall                     0.599680
potential                   0.519599
reactions                   0.506014
cm                          0.460179
lcm                         0.460179
rcm                         0.460179
st                          0.422810
ls                          0.422810
rs                          0.422810
lm                          0.414217
rm                          0.414217
ram                         0.414167
lam                         0.414167
cam                         0.414167
cf                          0.409604
rf                          0.409604
Name: eur_wage, dtype: float64

# Analyze

We try to use Linear Regression to detemine the salary base on the first 20 highest correlation features.

In [16]:
from sklearn import linear_model
lin_reg = linear_model.LinearRegression()
features = ['eur_release_clause', 'eur_value','international_reputation','overall','potential','reactions','cm','lcm','rcm','st','ls','rs','lm','rm','ram','lam','cam','cf','rf']
X = X_train[features]
lin_reg.fit(X,y_train)

NameError: name 'X_train' is not defined

In [ ]:
import numpy as np
prediction = lin_reg.predict(X_test[fea])

#remove "remove" this one when finish.
remove = (prediction - y_test)*100/y_test

# Only get right answer 220/3556
remove[np.abs(remove)<10].count()

It is clear that Linear Regression using only Pace, Shot, Pass, Drift, Defense, Physical features is not sufficient to correctly determining salary value.

In [ ]:
under_20000 = data[data['eur_wage']<= 20000]
find = under_20000[under_20000['overall']>80]
find